hummingage -- a fast and transparent Bayesian Age Depth Model
==================

Age-depth models are used in paleoclimate analyses in the field of reconstructing past climate environments. Vertical sediment profiles (cores) contain information about past environmental conditions at every depth. However, the relation between the depth in a sediment core and the corresponding age of the material is not given at any depth, because of the limited sampling rate (e.g. every 20 cm). To draw age conclusions also from depths inbetween sampling points a way of interpolating the samples is needed, hence the need for age-depth models. Additionally, and similar important (maybe even more) is a good estimation of the errors of the interpolated age-depth relation.

Similar to other methods we apply restrictions to our model, e.g. that age must increase monotonically with depth (e.g. Blaauw and Christen 2005). Further we adopt the idea of an underlying physical accumulation process of the sediment according to a sedimenatation rate (Blaauw and Christen, 2011). The evolution or change of the sedimentation rate over time is responsible for the shape of the age-depth curve.

If there are some good age-depth model on the market, the question arises, why do we need another one. The answer is twofold. (i) our aim is to provide a simple age-depth model, where the user can really get into the source code and adjust it to his/her needs. We don't want a black-box, where the user cannot interact with, unless being a programming and statistics expert. This includes a clear, simple and transparent error estimation, which is crucial. (ii) we apply a new Bayesian step (borrowed from astro-physics), not used yet in this way in age-depth modelling, which finally gives a very attractive answer to the essential question of finding the best fitting curve for the data given the physical basis and restrictions.

Input data
--------------------------------------

As input data for our method we require a strict text format:

"labID","age","error","depth"<br>
1,2.385,1.231,3<br>
2,9.960,1.346,43<br>
3,13.258,1.158,67<br>
4,15.391,1.140,93<br>
5,16.857,1.158,149<br>
6,20.172,1.254,203<br>
7,18.314,1.236,249<br>
8,19.498,1.369,313<br>
9,22.724,1.292,491<br>
10,25.007,1.379,515<br>
11,26.612,1.524,662<br>
12,27.054,2.436,740<br>

The first line must contain the names (exactly as seen in the above example) of the columns, whereas "labID" is simply a running number, "age" is the age of the data (in ka), the "error" is interpreted as a standard deviation and finally we have the depth (in cm). The units must not be ka and cm, however the plots are expecting these units. If you have other units, just change the labels of the plot axes.

The source code of the pure R script and the here shown R code (in Jupyter Notebook) is synchronized. However, the only difference is that we use the script as a function in pure R and as a script here in the Jupyter Notebook. Thus the pure R scripts start like below:

In [10]:
hummingage <- function(path,infile,segments,DiscardOutliers,EstimateNullRate,token,plot_format) {
    ##                                                                                                                                          
    ## usage:                                                                                                                                   
    ## hummingage("/home/user/data/","myfile.csv",c(NaN),TRUE,"estimate","KJmpmGgQ","png,pdf")                                                  
    ## hummingage("/home/user/data/","myfile.csv",c(124),FALSE,"estimate","KJmpmGgQ","png")                                                     
    ## hummingage("/home/user/data/","myfile.csv",c(334,856,1223),TRUE,"estimate","KJmpmGgQ","pdf")                                             
    ## segments in cm                                                                                                                           

    ## path: (string): is the path to the infile                                                                                                
    ## infile: (string): is the data file according to a certain format                                                                         
    ## segments: (integer array): is an array indicating roughly the positions of the segments, use c(NaN) for no segments                      
    ## DiscardOutliers: (boolean): can be TRUE or FALSE. If true, a simple algorithm is applied to identify outliers                            
    ## EstimateNullRate: (string or integer): possible values are                                                                               
    ##                   "estimate": the x-axis offset is estimated from the data                                                               
    ##                   "null": the x-axis offset is set to 0                                                                                  
    ##                   negative or positive integer: x-axis offset is set to the inserted value, e.g. -42, 14, ... (in cm)                    
    ## token: (string): this is a random string, which can be choosen arbitrarily                                                               
    ## plot_format (string): this can be "png,pdf", "png" or "pdf" and defines what kinds of images are written to disk                         
    ##                       both, png and pdf or only png or pdf                                                                               
    ##  
    ...
    }

whereas here in the Notebook we start with the code below. The "path" is simply the path to our data and "infile" holds the name of our data file. "segments" is crucial, while it defines in how many segments we split our data. More details are given later, here we just need to know that segments is an array. We use "c(NaN)" if do not want to split the data. We use "c(103)" if we want to split the data between the sample point before depth=103 cm and after depth=103 cm. The same would be accomplished by "c(105)", so segments do not need to be exact. If we would have used "c(103,345,564)" we would have split the data into four segments, i.e. cutting three times. As said above, we restrict the age data to increase monotonically with depth, thus negative accumulation rates are not allowed. A simple algorithm can be used to remove negative rates, i.e. outliers from the data by setting "DiscardOutliers=TRUE". It is important to note that it is recommended to quality control the data before to remove data by expert knowledge. The parameter "plot_format" is used to define the output format of the plots, whereas "plot_format="png,pdf"" yields both png and pdf, other possibilities are "plot_format="png"" and "plot_format="pdf"".

In [4]:
path <- "/home/jovyan/work/hummingage/"
infile <- "my_test_data_1.csv"
segments <- c(NaN)
DiscardOutliers <- FALSE
EstimateNullRate <- "estimate"
token <- "KHzvfrTgfR"
plot_format <- "png,pdf"
                                                                                                                   

    ## path: (string): is the path to the infile                                                                                                
    ## infile: (string): is the data file according to a certain format                                                                         
    ## segments: (integer array): is an array indicating roughly the positions of the segments, use c(NaN) for no segments                      
    ## DiscardOutliers: (boolean): can be TRUE or FALSE. If true, a simple algorithm is applied to identify outliers                            
    ## EstimateNullRate: (string or integer): possible values are                                                                               
    ##                   "estimate": the x-axis offset is estimated from the data                                                               
    ##                   "null": the x-axis offset is set to 0                                                                                  
    ##                   negative or positive integer: x-axis offset is set to the inserted value, e.g. -42, 14, ... (in cm)                    
    ## token: (string): this is a random string, which can be choosen arbitrarily                                                               
    ## plot_format (string): this can be "png,pdf", "png" or "pdf" and defines what kinds of images are written to disk                         
    ##                       both, png and pdf or only png or pdf                                                                               
    ##  

Next, we load the data and define some setting:

In [5]:
## full file-path                                                                                                                                                                        
    xfile <- paste(path,infile,sep="");

    ## transform plot_format to array                                                                                                                                                        
    plot_format_tmp <- strsplit(plot_format,",")
    plot_format <- plot_format_tmp[[1]]


    ## load the infile and save as ageDataOrig                                                                                                                                               
    ageDataOrig <- read.table(xfile, header = TRUE,sep=",")

    ## if no segment will be used set it to empty                                                                                                                                            
    if (is.nan(segments)){
        segments = c()
    }

    ## calculate length and estimate rates                                                                                                                                                   
    lengthData <- length(ageDataOrig$depth)
    diffDepths <- diff(ageDataOrig$depth)
    accRatesOrig <- diff(ageDataOrig$age)/diffDepths
